In [7]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50

from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
BATCH_SIZE = 64
EPOCHS = 5

In [9]:
dataset_name = 'patch_camelyon'
(train_dataset, validation_dataset), info = tfds.load(name=dataset_name, split=('train[:80%]', 'train[-20%:]'), with_info=True)

In [ ]:
len(list(train_dataset)), len(list(validation_dataset))

(209715, 52429)

In [10]:
def normalize(images):
    # 0~1 사이의 값으로 Normalize 합니다.
    img, lbl = tf.cast(images['image'], tf.float32) / 255.0, images['label']
    # 300 X 300 사이즈로 resize 합니다.
    img = tf.image.resize(img, size=(300, 300))
    return img, lbl

In [11]:
train_data = train_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)
valid_data = validation_dataset.map(normalize).batch(BATCH_SIZE).repeat().prefetch(1)

In [12]:
steps_per_epoch= int(len(list(train_dataset)) * 0.8) // BATCH_SIZE + 1
validation_steps= int(len(list(validation_dataset)) * 0.2) // BATCH_SIZE + 1

In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2), 
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2), 
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2), 
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2), 
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2), 
    Flatten(), 
    Dense(512, activation='relu'), 
    Dense(32, activation='relu'), 
    Dense(1, 'sigmoid')
])

In [ ]:
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_history = model_cnn.fit(train_data, 
                            validation_data=(valid_data),
                            steps_per_epoch=steps_per_epoch,
                            validation_steps=validation_steps,
                            epochs=5, 
                           )

Epoch 1/5
   2/2622 [..............................] - ETA: 5:14 - loss: 0.7059 - accuracy: 0.5469WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0376s vs `on_train_batch_end` time: 0.0898s). Check your callbacks.


2622/2622 [==============================] - 324s 123ms/step - loss: 0.4413 - accuracy: 0.7971 - val_loss: 0.3507 - val_accuracy: 0.8521
Epoch 2/5
2622/2622 [==============================] - 313s 120ms/step - loss: 0.3150 - accuracy: 0.8661 - val_loss: 0.2960 - val_accuracy: 0.8736
Epoch 3/5
2622/2622 [==============================] - 306s 117ms/step - loss: 0.2556 - accuracy: 0.8953 - val_loss: 0.2394 - val_accuracy: 0.9047
Epoch 4/5
2622/2622 [==============================] - 306s 117ms/step - loss: 0.2218 - accuracy: 0.9121 - val_loss: 0.2215 - val_accuracy: 0.9151
Epoch 5/5
2622/2622 [==============================] - 306s 117ms/step - loss: 0.1971 - accuracy: 0.9233 - val_loss: 0.2252 - val_accuracy: 0.9149


In [ ]:
# vgg 16

In [ ]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
transfer_model.trainable=False

In [ ]:
model_tr = Sequential([
    transfer_model, 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dense(1, 'sigmoid')
])

In [ ]:
model_tr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
transfer_history = model_tr.fit(train_data, 
                             validation_data=(valid_data),
                             steps_per_epoch=steps_per_epoch,
                             validation_steps=validation_steps,
                             epochs=5, 
                            )

Epoch 1/5
2622/2622 [==============================] - 1296s 494ms/step - loss: 0.3654 - accuracy: 0.8426 - val_loss: 0.3226 - val_accuracy: 0.8676
Epoch 2/5
2622/2622 [==============================] - 1300s 496ms/step - loss: 0.3129 - accuracy: 0.8695 - val_loss: 0.3446 - val_accuracy: 0.8516
Epoch 3/5
2622/2622 [==============================] - 1298s 495ms/step - loss: 0.2926 - accuracy: 0.8788 - val_loss: 0.3312 - val_accuracy: 0.8546
Epoch 4/5
2622/2622 [==============================] - 1297s 495ms/step - loss: 0.2769 - accuracy: 0.8853 - val_loss: 0.3206 - val_accuracy: 0.8665
Epoch 5/5
2622/2622 [==============================] - 1297s 495ms/step - loss: 0.2664 - accuracy: 0.8899 - val_loss: 0.2928 - val_accuracy: 0.8812


In [31]:
# vgg19

In [32]:
transfer_model = VGG19(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

In [33]:
transfer_model.trainable=False

In [34]:
model_tr = Sequential([
    transfer_model, 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dense(1, 'sigmoid')
])

In [35]:
model_tr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
transfer_history = model_tr.fit(train_data, 
                             validation_data=(valid_data),
                             steps_per_epoch=steps_per_epoch,
                             validation_steps=validation_steps,
                             epochs=5, 
                            )

Epoch 1/5
2622/2622 [==============================] - 1650s 629ms/step - loss: 0.3703 - accuracy: 0.8377 - val_loss: 0.3237 - val_accuracy: 0.8647
Epoch 2/5
2622/2622 [==============================] - 1660s 633ms/step - loss: 0.3141 - accuracy: 0.8665 - val_loss: 0.3394 - val_accuracy: 0.8518
Epoch 3/5
2622/2622 [==============================] - 1652s 630ms/step - loss: 0.2880 - accuracy: 0.8800 - val_loss: 0.2989 - val_accuracy: 0.8746
Epoch 4/5
2622/2622 [==============================] - 1650s 629ms/step - loss: 0.2697 - accuracy: 0.8882 - val_loss: 0.2922 - val_accuracy: 0.8800
Epoch 5/5
2622/2622 [==============================] - 1650s 629ms/step - loss: 0.2560 - accuracy: 0.8952 - val_loss: 0.3010 - val_accuracy: 0.8846


In [18]:
# resnet

In [29]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

In [26]:
model_tr = Sequential([
    transfer_model, 
    Flatten(), 
    Dense(64, activation='relu'), 
    Dense(1, 'sigmoid')
])

In [27]:
model_tr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
transfer_history = model_tr.fit(train_data, 
                             validation_data=(valid_data),
                             steps_per_epoch=steps_per_epoch,
                             validation_steps=validation_steps,
                             epochs=5, 
                            )

Epoch 1/5


ValueError: ignored